# Not a Lawyer | Remote

Using: 
* Pinecone (https://www.pinecone.io/) as vectorstore
* OpenAI Embeddings
* OpenAI API as LLM

Before you start, you will need to set the following environmental variables on your computer:

**Pinecone**
* PINECONE_API_KEY
* PINECONE_ENVIRONMENT
* PINECONE_INDEX

**OpenAI**
* OPENAI_API_KEY

----

### Step 1: Import Libraries

In [28]:
! pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 5.9 MB/s eta 0:00:0000:0100:01


In [29]:
# Import Libraries
import os 
from langchain.retrievers.multi_query import MultiQueryRetriever

from langchain.document_loaders import UnstructuredXMLLoader, TextLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter, HTMLHeaderTextSplitter

from langchain.vectorstores import Pinecone

from langchain.chains import RetrievalQA   

from langchain.prompts import PromptTemplate 

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


import gradio as gr



### Step 2: Define llm and embedding models

In [30]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

In [31]:
embedding = OpenAIEmbeddings()


### Step 3: Process Data & Set up Vector Database with Pinecone

#### Step 3.1: Define the data (URLs in this case) for the vector database

In [32]:
# German Residence Laws

aufentv =  "https://www.gesetze-im-internet.de/aufenthv/BJNR294510004.html"
aufenthg = "https://www.gesetze-im-internet.de/aufenthg_2004/BJNR195010004.html"
urls = [aufentv, aufenthg]

#### Step 3.2: Split by HTML Headers

In [33]:
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

docs = []

for url in urls:
    html_header_splits = html_splitter.split_text_from_url(url)
    docs += html_header_splits


chunk_size = 1000
chunk_overlap = 200
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(html_header_splits)


In [101]:
# Sanity Check: are there any splits (this should be non-zero)
len(splits)

785

#### Step 3.3: Create a vector database with Pinecone
You need to create a pinecone account, and create an index: https://app.pinecone.io

In [ ]:
# First time here? Upsert the documents to the vector store (you only need to do this once)

# vectorstore = Pinecone.from_documents(
#     documents=splits, embedding=OpenAIEmbeddings(), index_name="lawyer"
# )


In [35]:
# Do you already have your documents in the vector store? Then just load it:
vectorstore = Pinecone.from_existing_index("lawyer", OpenAIEmbeddings())

In [100]:
# Connect your retriever to the vector store
retriever = vectorstore.as_retriever()

-----

### Step 4: Set up the Prompt

#### Step 4.1: Define the prompt template

In [92]:
template = """
        ###INSTRUCTIONS: 
        You are polite and professional question-answering AI assistant. You must provide a helpful response to the user. 
        
        In your response, PLEASE ALWAYS:
          (0) Be a detail-oriented reader: read the question and context and understand both before answering
          (1) Start your answer with a friendly tone, and reiterate the question so the user is sure you understood it
          (2) If the context enables you to answer the question, write a detailed, helpful, and easily understandable answer with sources referenced inline. IF NOT: you can't find the answer, respond with an explanation, starting with: "I couldn't find the information in the laws I have access to". 
          (3) Below the answer, please list out all the referenced sources (i.e. legal paragraphs backing up your claims)
          (4) Now you have your answer, that's amazing - review your answer to make sure it answers the question, is helpful and professional and formatted to be easily readable.
        
        Think step by step. 
        ###
        
      Answer the following question using the context provided.
        ### Question: {question} ###

        ### Context: {context} ###

        

        ### Helpful Answer with Sources:

        """

    # create prompt template
prompt = PromptTemplate.from_template(template)

#### Step 4.2: Create the Chain

In [93]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Check that it's working by running the following cell:

In [94]:
ans = chain.invoke("How do I get a blue card in Germany?")

print(ans)

Hello! To get a Blue Card in Germany, you will need to have a valid job offer or employment contract for a position that is suitable for someone with a university degree or equivalent tertiary education. You will also need to have a Blue Card from a non-Schengen EU member state, and you must carry this card with you upon entry into Germany. Additionally, you will need to have adequate housing available and meet the necessary language requirements.

If you have been employed under the Blue Card scheme for at least 33 months and have made contributions to the statutory pension insurance during this time, you may be eligible for a settlement permit. This permit can also be granted if you have been employed for 21 months and have sufficient knowledge of the German language.

It's important to note that the specific requirements and procedures for obtaining a Blue Card may vary, so it's advisable to consult with the relevant German authorities or an immigration lawyer for personalized guida

-----

### Step 5: Set up Simple UI using Gradio

#### Step 5.1: Create a function to use in Gradio

In [95]:
# Creating a function that takes a question and returns an answer is a good idea since 
# I'll make a gradio UI in the next cell, and this simplifies it
def get_answer(question):
    answer = chain.invoke(question)
    return answer

#### Step 5.2: Create and run the Gradio interface

In [102]:

iface = gr.Interface(fn=get_answer, inputs=gr.Textbox(
                    value="Enter your question"), 
                     outputs="markdown",  
                     title="LLM Augmented Q&A over German Residence Laws",
                     description="Ask a question about German Residence Laws and get an answer from a friendly AI assistant. This assistant looks up relevant German Residence laws and answers your question.",
                     examples=[["How do I get a blue card in Germany?"], 
                               ["How long can I stay in Germany with a tourist visa?"],
                               ["How do I get a work visa in Germany?"],["I want to work in Germany, what visa do I need?"],
                               ["I am a student in Germany, can I work?"]],
                    theme=gr.themes.Soft(),
                    allow_flagging="never",)

iface.launch()

Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.
